# Analysis Notebook - Count Genes and Events

This notebook processes the raw counts as provided by rMATS and performs some descriptive statistical analysis. It is used to produce the following outputs. 

## Data files created by this notebook
Output text files are written to the ``data/`` directory (at the same level as the ``jupyter`` directory). 

1. **gene_as.tsv**: Alternative splicing events per gene, adjusted p-value <= 0.05, fold change >= 1.5.
2. **all_gene_as.tsv**: all alternative splicing events
3. **gene_dge.tsv**: Differential gene expression, adjusted p-value <= 0.05, fold change >= 1.5
4. **genesWithCommonAS.tsv**: genes (as geneSymbol, the number of splicing events, and the number of tissues the event occurs in)
5. **Total_AS_by_chr.tsv**: Total alternative splicing events per chromosome
6. **Total_AS_by_geneSymbol.tsv**: Count the number of tissues in which specific genes show significant alternative splicing
7. **DGE_by_geneSymbol.tsv***: Most highly expressed genes by tissue
5. **Total_AS_by_tissue.tsv**: Count the number of significant splicing events per tissue
6. **Total_AS_by_splicingtype.tsv**: Count number of significant splicing events for each of the 5' alternative splicing categories
7. **SplicingIndex_chr.tsv**: Splicing index by chr (number of sigificant AS events per 1000 exons)

In [8]:
defaultW <- getOption("warn")  # suppress warnings for this cell
options(warn = -1) 
library(dplyr)
library(ggplot2)
library(limma)
library(multtest)
library(Biobase)
library(edgeR)
library(tibble)
library(R.utils)
library(rtracklayer)

options(warn = defaultW)

## 1. Download all the rMATS results

Each of the alternative splicing output files are downloaded here:

### 1.1 get released rMATS GTF annotations

For each splicing type, the junctions are defined, so we have 5 specific annotated splicing specific junction ID annotation files:

1. **fromGTF.A3SS.txt**: annotations for the alternative 3' splice site junctions
2. **fromGTF.A5SS.txt**: annotations for the alternative 5' splice site junctions
3. **fromGTF.MXE.txt**: annotations for the mutually exclusive exon junctions
4. **fromGTF.RI.txt**: annotations for the retained introns junctions
5. **fromGTF.SE.txt**: annotations for the skipped exon junctions

### 1.2 Unpack the data.tar file if necessary
To run this script, we need to import three compressed files and unpack them.

| file | sha256 | filename  |
|------|------  |-----------|
|  1   | b0c4bb23b96d77aba7e731fa2a15dc74a34daf490312478aca94443f9a6d4e90 | results/data_as_dge.tar.gz |
|  2   | a0c2c5a7d7cfa0a89c8a39e2f7a4c6c3ac8c6a860f721077c087614505d869cf | rmats_and_annotation.tar.gz |


In [9]:
data_as_dge_file_dir <- list.files("../../mounted-data", pattern='data_as_dge.tar.gz')
data_as_dge_file_dir
rmats_and_annotation_dir <- list.files("../../mounted-data", pattern='rmats_and_annotation.tar.gz')
rmats_and_annotation_dir

[1] "5ef252b5143fa0011321ec40-data_as_dge.tar.gz-5ef252b5143fa0011321ec40"

[1] "5ef252a9143fa0011321ec01-rmats_and_annotation.tar.gz-5ef252a9143fa0011321ec01"

In [10]:
data_as_dge_file_dir <- list.files("../../mounted-data", pattern='data_as_dge.tar.gz')
data_as_dge_file <- paste("../../mounted-data", data_as_dge_file_dir, 'robinson-bucket/notebooks/data_as_dge', sep='/')
data_as_dge_file_tar_gz <- paste(data_as_dge_file, '.tar.gz', sep='')
message("In order to unpack the necessary files, execute the following commands on the shell.")
message("data_as_dge.tar.gz")
mycommand = paste("tar xvfz ",data_as_dge_file_tar_gz, "-C ../data", sep=" ")
message(mycommand)
system(mycommand)
message("checking sha256sum")
mycommand = paste("sha256sum", data_as_dge_file_tar_gz, sep = " ")
message(mycommand)
system(mycommand)
rmats_annot_file_dir <- list.files("../../mounted-data", pattern='rmats_and_annotation.tar.gz')
rmats_annot_file <- paste("../../mounted-data", rmats_annot_file_dir, 'robinson-bucket/notebooks/rmats_and_annotation', sep='/')
rmats_annot_file_tar_gz <- paste(rmats_annot_file, '.tar.gz', sep='')
message("In order to unpack the necessary files, execute the following commands on the shell.")
message("rmats_and_annotation.tar.gz")
mycommand = paste("tar xvfz ",rmats_annot_file_tar_gz, "-C ../data", sep=" ")
message(mycommand)
system(mycommand)
message("checking sha256sum")
mycommand = paste("sha256sum", rmats_annot_file_tar_gz, sep = " ")
message(mycommand)
system(mycommand)

In order to unpack the necessary files, execute the following commands on the shell.

data_as_dge.tar.gz

tar xvfz  ../../mounted-data/5ef252b5143fa0011321ec40-data_as_dge.tar.gz-5ef252b5143fa0011321ec40/robinson-bucket/notebooks/data_as_dge.tar.gz -C ../data

checking sha256sum

sha256sum ../../mounted-data/5ef252b5143fa0011321ec40-data_as_dge.tar.gz-5ef252b5143fa0011321ec40/robinson-bucket/notebooks/data_as_dge.tar.gz

In order to unpack the necessary files, execute the following commands on the shell.

rmats_and_annotation.tar.gz

tar xvfz  ../../mounted-data/5ef252a9143fa0011321ec01-rmats_and_annotation.tar.gz-5ef252a9143fa0011321ec01/robinson-bucket/notebooks/rmats_and_annotation.tar.gz -C ../data

checking sha256sum

sha256sum ../../mounted-data/5ef252a9143fa0011321ec01-rmats_and_annotation.tar.gz-5ef252a9143fa0011321ec01/robinson-bucket/notebooks/rmats_and_annotation.tar.gz



In [11]:
## get the rmats 3.2.5 discovered/annotated junction information in GTF format
message("Decompressing fromGTF.tar.gz into ../data")
system("mkdir -p ../data && tar xvfz ../data/fromGTF.tar.gz -C ../data", intern = TRUE)
system("gunzip ../data/fromGTF.*txt.gz", intern = TRUE)
message("Done!\n")

Decompressing fromGTF.tar.gz into ../data



[1] "fromGTF.A3SS.txt.gz"             "fromGTF.A5SS.txt.gz"            
 [3] "fromGTF.MXE.txt.gz"              "fromGTF.RI.txt.gz"              
 [5] "fromGTF.SE.txt.gz"               "fromGTF.novelEvents.A3SS.txt.gz"
 [7] "fromGTF.novelEvents.A5SS.txt.gz" "fromGTF.novelEvents.MXE.txt.gz" 
 [9] "fromGTF.novelEvents.RI.txt.gz"   "fromGTF.novelEvents.SE.txt.gz"

Warning message in system("gunzip ../data/fromGTF.*txt.gz", intern = TRUE):
“running command 'gunzip ../data/fromGTF.*txt.gz' had status 2”


character(0)
attr(,"status")
[1] 2
attr(,"errmsg")
[1] "Resource temporarily unavailable"

Done!




### 2  Refined results
We define **refined results** as (FC >= 1.5 and pVal <= 0.05) for the sex\*as_event coefficient result for the linear model

### 2.1 getTissueReduction

In [12]:
tissue_reduction_filename <- "../assets/tissues.tsv"
tissue_reduction <- read.table(tissue_reduction_filename, header=TRUE, sep="\t",
                               skipNul=FALSE, stringsAsFactors = FALSE)
colnames(tissue_reduction)  <- c("SMTSD","female","male","include","display_name")
tissue_reduction <- tissue_reduction[tissue_reduction$display_name != "n/a",]
tissue_reduction$display_name <- factor(tissue_reduction$display_name)
levels(tissue_reduction$display_name)
message("We extracted ", length(levels(tissue_reduction$display_name))," different tissues with at least 50 samples in both M & f")

[1] "Adipose (sc)"          "Adipose (v)"           "Adrenal gland"        
 [4] "Aorta"                 "Atrial appendage"      "Breast"               
 [7] "Caudate"               "Cerebellar hemisphere" "Cerebellum"           
[10] "Coronary artery"       "Cortex"                "EBV-lymphocytes"      
[13] "Esophagus (gej)"       "Esophagus (m)"         "Esophagus (mu)"       
[16] "Fibroblasts"           "Frontal cortex"        "Hippocampus"          
[19] "Hypothalamus"          "Left ventricle"        "Liver"                
[22] "Lung"                  "Nucleus accumbens"     "Pancreas"             
[25] "Pituitary"             "Putamen"               "Sigmoid colon"        
[28] "Skeletal muscle"       "Skin (exposed)"        "Skin (not exposed)"   
[31] "Small intestine"       "Spinal cord"           "Spleen"               
[34] "Stomach"               "Thyroid"               "Tibial artery"        
[37] "Tibial nerve"          "Transverse colon"      "Whole blood"

We extracted 39 different tissues with at least 50 samples in both M & f



### 2.2 Read in refined results and annotations

In [13]:
significant_results_dir = "../data/"
pattern = "model_B_sex_as_events_refined.csv"
files <- list.files(path = significant_results_dir, pattern = pattern)
as_types <- c("a3ss", "a5ss", "mxe", "ri", "se")
length(files)

[1] 195

In [14]:
a3ss_annot <- read.table(file = "../data/fromGTF.A3SS.txt", sep = "\t", quote = "\"", header = T, stringsAsFactors = F)
a5ss_annot <- read.table(file = "../data/fromGTF.A5SS.txt", sep = "\t", quote = "\"", header = T, stringsAsFactors = F)
mxe_annot <- read.table(file = "../data/fromGTF.MXE.txt", sep = "\t", quote = "\"", header = T, stringsAsFactors = F)
ri_annot <- read.table(file = "../data/fromGTF.RI.txt", sep = "\t", quote = "\"", header = T, stringsAsFactors = F)
se_annot <- read.table(file = "../data/fromGTF.SE.txt", sep = "\t", quote = "\"", header = T, stringsAsFactors = F)

In [15]:
head(se_annot)

,ID,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,ENSG00000034152.18,MAP2K3,chr17,+,21287990,21288091,21284709,21284969,21295674,21295769
2,2,ENSG00000034152.18,MAP2K3,chr17,+,21303182,21303234,21302142,21302259,21304425,21304553
3,3,ENSG00000034152.18,MAP2K3,chr17,+,21295674,21295769,21287990,21288091,21296085,21296143
4,4,ENSG00000034152.18,MAP2K3,chr17,+,21295674,21295769,21287990,21288091,21298412,21298479
5,5,ENSG00000034152.18,MAP2K3,chr17,+,21295674,21295769,21284710,21284969,21296085,21296143
6,6,ENSG00000034152.18,MAP2K3,chr17,+,21295674,21295769,21284710,21284969,21298412,21298479


### 2.3 create_as_structure 

This function doees an aggregation of the alternative splicing events - good for all events and the significantly expressed events.

In [16]:
create_as_structure <- function ( results_dir, files, pattern, tissue_reduction) {
    gene_as = data.frame()
    counts <- rep(NA, length(files))
    message("\nnumber of files:", paste(length(files)), collapse = "")
    for (i in 1:length(files)) {
       lines  <- read.table(file=paste0(results_dir, files[i]), 
                                     header = TRUE, sep = ",", quote = "\"'", skipNul = FALSE)
       if (dim(lines)[1] > 0) {
           event     <- as.vector(as.character(rownames(lines)))
           tissue1   <- gsub(pattern,"", files[i], fixed = TRUE)
           counts[i] <- dim(lines)[1]
           event_idx <- substring(event, regexpr("[0-9]+$", event))
           res       <- data.frame()
           if (grepl("^a3ss_", files[i])) {
               # remove the first 5 letters of the string 
               tissue2 <- substring(tissue1,6)
               idx <- match(event_idx, a3ss_annot$ID)
               res <- data.frame(GeneJunction <- event,
                              ASE          <- "A3SS", 
                              ASE_IDX      <- idx,
                              Tissue       <- tissue2,
                              counts       <- counts[i],
                              Display      <- tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"],
                              GeneSymbol   <- a3ss_annot$geneSymbol[idx],
                              GeneID       <- a3ss_annot$GeneID[idx],
                              chr          <- a3ss_annot$chr[idx],
                              logFC        <- lines$logFC,
                              AveExpr      <- lines$AveExpr,
                              t            <- lines$t,
                              PValue       <- lines$P.Value,
                              AdjPVal      <- lines$adj.P.Val,
                              B            <- lines$B)
               colnames(res) <- c("GeneJunction","ASE","ASE_IDX","Tissue","counts","Display",
                                  "GeneSymbol","GeneID","chr","logFC","AveExpr","t","PValue","AdjPVal","B")
               gene_as <- rbind(gene_as,res)
            
           } else if (grepl("^a5ss_", files[i])) {
               # remove the first 5 letters of the string 
               tissue2 <- substring(tissue1,6)
               idx <- match(event_idx, a5ss_annot$ID)
               res <- data.frame(GeneJunction <- event,
                              ASE          <- "A5SS", 
                              ASE_IDX      <- idx,
                              Tissue       <- tissue2,
                              counts       <- counts[i],
                              Display      <- tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"],
                              GeneSymbol   <- a5ss_annot$geneSymbol[idx],
                              GeneID       <- a5ss_annot$GeneID[idx],
                              chr          <- a5ss_annot$chr[idx],
                              logFC        <- lines$logFC,
                              AveExpr      <- lines$AveExpr,
                              t            <- lines$t,
                              PValue       <- lines$P.Value,
                              AdjPVal      <- lines$adj.P.Val,
                              B            <- lines$B)
               colnames(res) <- c("GeneJunction","ASE","ASE_IDX","Tissue","counts","Display",
                               "GeneSymbol","GeneID","chr","logFC","AveExpr","t","PValue","AdjPVal","B")
               gene_as <- rbind(gene_as,res)
           } else if (grepl("^mxe_", files[i])) {
               # remove the first 4 letters of the string 
               tissue2 <- substring(tissue1,5)
               idx <- match(event_idx, a3ss_annot$ID)
               res <- data.frame(GeneJunction <- event,
                              ASE          <- "MXE", 
                              ASE_IDX      <- idx,
                              Tissue       <- tissue2,
                              counts       <- counts[i],
                              Display      <- tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"],
                              GeneSymbol   <- mxe_annot$geneSymbol[idx],
                              GeneID       <- mxe_annot$GeneID[idx],
                              chr          <- mxe_annot$chr[idx],
                              logFC        <- lines$logFC,
                              AveExpr      <- lines$AveExpr,
                              t            <- lines$t,
                              PValue       <- lines$P.Value,
                              AdjPVal      <- lines$adj.P.Val,
                              B            <- lines$B)
               colnames(res) <- c("GeneJunction","ASE","ASE_IDX","Tissue","counts","Display",
                                  "GeneSymbol","GeneID","chr","logFC","AveExpr","t","PValue","AdjPVal","B")
               gene_as <- rbind(gene_as,res)
           } else if (grepl("^se_", files[i])) {
               # remove the first 3 letters of the string 
               tissue2 <- substring(tissue1,4)
               idx <- match(event_idx, se_annot$ID)
               res <- data.frame(GeneJunction <- event,
                              ASE          <- "SE", 
                              ASE_IDX      <- idx,
                              Tissue       <- tissue2,
                              counts       <- counts[i],
                              Display      <- tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"],
                              GeneSymbol   <- se_annot$geneSymbol[idx],
                              GeneID       <- se_annot$GeneID[idx],
                              chr          <- se_annot$chr[idx],
                              logFC        <- lines$logFC,
                              AveExpr      <- lines$AveExpr,
                              t            <- lines$t,
                              PValue       <- lines$P.Value,
                              AdjPVal      <- lines$adj.P.Val,
                              B            <- lines$B)
               colnames(res) <- c("GeneJunction","ASE","ASE_IDX","Tissue","counts","Display",
                                  "GeneSymbol","GeneID","chr","logFC","AveExpr","t","PValue","AdjPVal","B")
               gene_as <- rbind(gene_as,res)
           } else if (grepl("^ri_", files[i])){
               # remove the first 3 letters of the string 
               tissue2 <- substring(tissue1,4)
               idx <- match(event_idx, ri_annot$ID)
               res <- data.frame(GeneJunction <- event,
                              ASE          <- "RI", 
                              ASE_IDX      <- idx,
                              Tissue       <- tissue2,
                              counts       <- counts[i],
                              Display      <- tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"],
                              GeneSymbol   <- ri_annot$geneSymbol[idx],
                              GeneID       <- ri_annot$GeneID[idx],
                              chr          <- ri_annot$chr[idx],
                              logFC        <- lines$logFC,
                              AveExpr      <- lines$AveExpr,
                              t            <- lines$t,
                              PValue       <- lines$P.Value,
                              AdjPVal      <- lines$adj.P.Val,
                              B            <- lines$B)
               colnames(res) <- c("GeneJunction","ASE","ASE_IDX","Tissue","counts","Display",
                                  "GeneSymbol","GeneID","chr","logFC","AveExpr","t","PValue","AdjPVal","B")
               gene_as <- rbind(gene_as,res)
           }
        
       } #if has sig. events
    
   } #for all files
   colnames(gene_as) <- c("GeneJunction","ASE","ASE_IDX","Tissue","counts","Display","GeneSymbol","GeneID","chr","logFC","AveExpr","t","PValue","AdjPVal","B")
   n_unique_genes <- length(summary(as.factor(gene_as$GeneSymbol),maxsum=50000))
   message("We extracted a total of ",nrow(gene_as)," significant alternative splicing events (gene_as)")
   message("This includes ", n_unique_genes, " total genes")
   return (gene_as)
}

### 2.4 create_dge_structure 

This function doees an aggregation of the differential gene expression events - good for all events and the significantly expressed events.

In [17]:
create_dge_structure <- function ( results_dir, files, pattern, map_pattern, tissue_reduction) {
   gene_dge = data.frame()
   counts <- rep(NA, length(files))
   for (i in 1:length(files)) {
      lines  <- read.table(file=paste0(results_dir, files[i]), 
                                     header = TRUE, sep = ",", quote = "\"'", skipNul = FALSE)
      if (dim(lines)[1] > 0) {
         tissue1    <- gsub(pattern,"", files[i], fixed = TRUE)
         map_lines  <- read.table(file=paste0(paste0(results_dir, tissue1),map_pattern),
                                     header = TRUE, sep = ",", quote = "\"'", skipNul = FALSE)
         counts[i]  <- dim(lines)[1]    
         tissue1    <- gsub(pattern,"", files[i], fixed = TRUE)
         map_lines  <- read.table(file=paste0(paste0(results_dir, tissue1),map_pattern),
                                     header = TRUE, sep = ",", quote = "\"'", skipNul = FALSE)
         ensg_ver   <- as.vector(as.character(rownames(lines)))
         ensg_no_ver<- as.vector(as.character(map_lines$ensg_names))
         ensg_genes <- as.vector(as.character(map_lines$ensg_genes))
         counts[i]  <- dim(lines)[1]  
         res <- data.frame(Tissue       <- tissue1,
                           ENSG_ver     <- ensg_ver,
                           ENSG_no_ver  <- ensg_no_ver,
                           GeneSymbol   <- ensg_genes,
                           counts       <- counts[i],
                           Display      <- tissue_reduction[tissue_reduction$SMTSD == tissue1, "display_name"],
                           logFC        <- lines$logFC,
                           AveExpr      <- lines$AveExpr,
                           t            <- lines$t,
                           PValue       <- lines$P.Value,
                           AdjPVal      <- lines$adj.P.Val,
                           B            <- lines$B)
         colnames(res) <- c("Tissue","ENSG_ver","ENSG_no_ver","GeneSymbol","counts","Display",
                            "logFC","AveExpr","t","PValue","AdjPVal","B")
         gene_dge <- rbind(gene_dge, res)
       } #if has sig. events
    } #for all files
    colnames(gene_dge) <- c("Tissue","ENSG_ver","ENSG_no_ver","GeneSymbol","counts","Display",
                        "logFC","AveExpr","t","PValue","AdjPVal","B")
    n_unique_genes <- length(summary(as.factor(gene_dge$GeneSymbol),maxsum=50000))
    message("We extracted a total of ",nrow(gene_dge)," significant differential gene events (gene_dge)")
    message("This includes ", n_unique_genes, " total genes")
    return(gene_dge)
}

### 2.5 Read in the alternative splicing results

We will create an aggregation of  all the results and all the significant results


In [18]:
results_dir         <- "../data/"
significant_pattern <- "_AS_model_B_sex_as_events_refined.csv"
significant_files   <- list.files(path = results_dir, pattern = significant_pattern)
all_pattern         <- "_AS_model_B_sex_as_events.csv"
all_files           <- list.files(path = results_dir, pattern = all_pattern)
as_types            <- c("a3ss", "a5ss", "mxe", "ri", "se")
length(all_files)
length(significant_files)

gene_as     <- create_as_structure (results_dir      <- results_dir, 
                                    files            <- significant_files, 
                                    pattern          <- significant_pattern, 
                                    tissue_reduction <- tissue_reduction)
all_gene_as <- create_as_structure (results_dir      <- results_dir, 
                                    files            <- all_files, 
                                    pattern          <- all_pattern, 
                                    tissue_reduction <- tissue_reduction)
head(gene_as,2)
head(all_gene_as, 2)
gene_as$Tissue <- factor(gene_as$Tissue)
write.table(gene_as, "../data/gene_as.tsv", quote=FALSE, sep="\t")
write.table(all_gene_as, "../data/all_gene_as.tsv", quote=FALSE, sep="\t")

[1] 195

[1] 195


number of files:195

We extracted a total of 6370 significant alternative splicing events (gene_as)

This includes 2887 total genes


number of files:195

We extracted a total of 2322562 significant alternative splicing events (gene_as)

This includes 14694 total genes



,GeneJunction,ASE,ASE_IDX,Tissue,counts,Display,GeneSymbol,GeneID,chr,logFC,AveExpr,t,PValue,AdjPVal,B
,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,XIST-2253,A3SS,2253,adipose_subcutaneous,4,Adipose (sc),XIST,ENSG00000229807.11,chrX,-4.408605,3.196317,-36.48897,4.635568e-154,3.893877e-150,310.0160
2,XIST-2252,A3SS,2252,adipose_subcutaneous,4,Adipose (sc),XIST,ENSG00000229807.11,chrX,-2.414713,3.647690,-21.92106,1.444102e-78,6.065229e-75,160.0282


,GeneJunction,ASE,ASE_IDX,Tissue,counts,Display,GeneSymbol,GeneID,chr,logFC,AveExpr,t,PValue,AdjPVal,B
,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,XIST-2253,A3SS,2253,adipose_subcutaneous,8400,Adipose (sc),XIST,ENSG00000229807.11,chrX,-4.408605,3.196317,-36.48897,4.635568e-154,3.893877e-150,310.0160
2,XIST-2252,A3SS,2252,adipose_subcutaneous,8400,Adipose (sc),XIST,ENSG00000229807.11,chrX,-2.414713,3.647690,-21.92106,1.444102e-78,6.065229e-75,160.0282


### 2.6 Create a genes-id file capturing the unique gene-junction locations in a single file
rMATS 3.2.5 unique junction ids by splicing event tied together with gene names and these identifiers useful for downstream analyses and investigations.

In [40]:
results_dir         <- "../data/"
significant_pattern <- "^se_*AS_model_B_sex_as_events_refined.csv"
files   <- list.files(path =results_dir, pattern = glob2rx(significant_pattern))
geneids <- data.frame()
i = 1
files[i]
pattern="_AS_model_B_sex_as_events_refined.csv"
for (i in 1:length(files)) {
    lines  <- read.table(file=paste0(results_dir, files[i]), 
                                     header = TRUE, sep = ",", quote = "\"'", skipNul = FALSE)
    
    if (dim(lines)[1] > 0) {
           event     <- as.vector(as.character(rownames(lines)))
           tissue1   <- gsub(pattern,"", files[i], fixed = TRUE)
           event_idx <- substring(event, regexpr("[0-9]+$", event))
           res       <- data.frame()
           tissue2 <- substring(tissue1,4)
           idx <- match(event_idx, se_annot$ID)
           res <- data.frame(geneIDs      <- event,
                             ID           <- event_idx,
                             GeneSymbol   <- se_annot$geneSymbol[idx],
                             GeneID       <- se_annot$GeneID[idx],
                             chr          <- se_annot$chr[idx])
           outfilename <- paste0(paste0("../data/se_",tissue2),"_geneids.tsv")
           write.table(res, outfilename, quote=FALSE, sep="\t")
           
     }
}

[1] "se_adipose_subcutaneous_AS_model_B_sex_as_events_refined.csv"

### 2.7 Read in the differential gene expression results

Here we create an aggregation of al the significant results differential gene expression events.

In [19]:
results_dir         <- "../data/"
significant_pattern <- "_DGE_refined.csv"
significant_files   <- list.files(path = results_dir, pattern = significant_pattern)
map_pattern         <- "_DGE_ensg_map.csv"
length(significant_files)

gene_dge     <- create_dge_structure (results_dir      <- results_dir, 
                                      files            <- significant_files, 
                                      pattern          <- significant_pattern, 
                                      map_pattern      <- map_pattern,
                                      tissue_reduction <- tissue_reduction)

head(gene_dge,2)

[1] 39

We extracted a total of 12633 significant differential gene events (gene_dge)

This includes 7417 total genes



,Tissue,ENSG_ver,ENSG_no_ver,GeneSymbol,counts,Display,logFC,AveExpr,t,PValue,AdjPVal,B
,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,adipose_subcutaneous,ENSG00000176728.7,ENSG00000176728,TTTY14,765,Adipose (sc),-7.982166,-0.9288129,-139.8230,0,0,1107.423
2,adipose_subcutaneous,ENSG00000231535.5,ENSG00000231535,LINC00278,765,Adipose (sc),-6.095420,-2.7765638,-126.9138,0,0,1050.366


Load in the gencode.v30.annotation.gtf file for additional annotation

In [20]:
#
# add chr information for summary data later, use the annotation we used for rMATS
#
if (!("gencode.v30.annotation.gtf.gz" %in% list.files("../data/"))) {
    message("downloading gencode v30 annotation\n")
    system("wget -O ../data/gencode.v30.annotation.gtf.gz ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_30/gencode.v30.annotation.gtf.gz")
    message("Done!\n")
    message("Unzipping compressed file gencode.v30.annotation.gtf.gz..")
    system("gunzip ../data/gencode.v30.annotation.gtf.gz", intern = TRUE)
    message("Done! gencode.v30.annotation.gtf can be found in ../data/")
}
gencode <- import("../data/gencode.v30.annotation.gtf")
gtf.df <- as.data.frame (gencode)
chr_genes <- unique(gtf.df[,c("seqnames","gene_name","gene_id")])
colnames(chr_genes) <- c("chr","GeneSymbol", "ENSG")
head(chr_genes)


,chr,GeneSymbol,ENSG
,<fct>,<chr>,<chr>
1,chr1,DDX11L1,ENSG00000223972.5
13,chr1,WASH7P,ENSG00000227232.5
26,chr1,MIR6859-1,ENSG00000278267.1
29,chr1,MIR1302-2HG,ENSG00000243485.5
37,chr1,MIR1302-2,ENSG00000284332.1
40,chr1,FAM138A,ENSG00000237613.2


In [21]:
for (i in 1:dim(chr_genes)[1]) {
    chr_genes$ENSG[i] <- as.character(strsplit(chr_genes$ENSG[i],'\\.\\w+$'))
}
head(chr_genes)

,chr,GeneSymbol,ENSG
,<fct>,<chr>,<chr>
1,chr1,DDX11L1,ENSG00000223972
13,chr1,WASH7P,ENSG00000227232
26,chr1,MIR6859-1,ENSG00000278267
29,chr1,MIR1302-2HG,ENSG00000243485
37,chr1,MIR1302-2,ENSG00000284332
40,chr1,FAM138A,ENSG00000237613


In [23]:
i = 1
chr <- rep("NA",dim(gene_dge)[1])
gene_dge$chr <- chr
for (i in 1:dim(gene_dge)[1]) {
    match  <- as.character(chr_genes$ENSG) %in% as.character((gene_dge$ENSG_no_ver[i]))
    if (sum(match==TRUE)== 1) {
        chr[i] <- as.character(chr_genes[match,]$chr)
        gene_dge$chr[i] <- chr[i]
    } else if (sum(match==TRUE)>1) {
        all <- as.vector(as.character(chr_genes[match,]$chr))
        gene_dge$chr[i] <- as.character(all[1])
    }
}
head(gene_dge)

write.table(gene_dge, "../data/gene_dge.tsv", quote=FALSE, sep="\t")

,Tissue,ENSG_ver,ENSG_no_ver,GeneSymbol,counts,Display,logFC,AveExpr,t,PValue,AdjPVal,B,chr
,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,adipose_subcutaneous,ENSG00000176728.7,ENSG00000176728,TTTY14,765,Adipose (sc),-7.982166,-0.9288129,-139.8230,0,0,1107.423,chrY
2,adipose_subcutaneous,ENSG00000231535.5,ENSG00000231535,LINC00278,765,Adipose (sc),-6.095420,-2.7765638,-126.9138,0,0,1050.366,chrY
3,adipose_subcutaneous,ENSG00000129824.15,ENSG00000129824,RPS4Y1,765,Adipose (sc),-9.664190,4.6352877,-125.8271,0,0,1041.877,chrY
4,adipose_subcutaneous,ENSG00000067646.11,ENSG00000067646,ZFY,765,Adipose (sc),-9.504590,0.6727555,-125.0371,0,0,1033.611,chrY
5,adipose_subcutaneous,ENSG00000229807.10,ENSG00000229807,XIST,765,Adipose (sc),9.892810,1.2375604,121.6969,0,0,1030.178,chrX
6,adipose_subcutaneous,ENSG00000229236.1,ENSG00000229236,TTTY10,765,Adipose (sc),-6.209013,-2.7452436,-122.5403,0,0,1029.414,chrY


### 2.8 Summary gene_as and gene_dge regarding events by tissue

In [47]:
XY <- gene_as %>% group_by(Tissue) %>% tally()
XY <- XY[order(XY$n),decreasing=TRUE]
head(XY)
message("Minimum splicing events per tissue ", min(XY$n), " maximum splicing events per tissue ", max(XY$n))
#XY[XY$n>100,]
sum(XY$n<100)

XY <- gene_dge %>% group_by(Tissue) %>% tally()
XY <- XY[order(XY$n),decreasing=TRUE]
head(XY)
message("Minimum gene expression events per tissue ", min(XY$n), " maximum gene expression events per tissue ", max(XY$n))
#XY[XY$n>100,]
sum(XY$n<100)

ERROR: Error in `[.tbl_df`(XY, order(XY$n), decreasing = TRUE): unused argument (decreasing = TRUE)


In [28]:
table(gtf.df[,c("gene_type")])



          3prime_overlapping_ncRNA                          antisense 
                               152                              46899 
     bidirectional_promoter_lncRNA                          IG_C_gene 
                              1655                                296 
                   IG_C_pseudogene                          IG_D_gene 
                                33                                152 
                         IG_J_gene                    IG_J_pseudogene 
                                76                                  9 
                     IG_pseudogene                          IG_V_gene 
                                 3                               1169 
                   IG_V_pseudogene                            lincRNA 
                               664                              65178 
                      macro_lncRNA                              miRNA 
                                 3                               5643 
     

### 3 Data Structures for Figures

### 3.1 gene_as.tsv

This file contains (description)
Here is a typical line
<pre>
A data.frame: 6 × 15
GeneJunction	ASE	ASE_IDX	Tissue	counts	Display	GeneSymbol	GeneID	chr	logFC	AveExpr	t	PValue	AdjPVal	B
<fct>	<fct>	<int>	<fct>	<int>	<fct>	<fct>	<fct>	<fct>	<dbl>	<dbl>	<dbl>	<dbl>	<dbl>	<dbl>
1	XIST-2253	A3SS	2253	adipose_subcutaneous	4	Adipose (sc)	XIST	ENSG00000229807.11	chrX	-4.4086049	3.196317	-36.488970	4.635568e-154	3.893877e-150	310.016049
2	XIST-2252	A3SS	2252	adipose_subcutaneous	4	Adipose (sc)	XIST	ENSG00000229807.11	chrX	-2.4147126	3.647690	-21.921057	1.444102e-78	6.065229e-75	160.028167
3	GREB1L-4933	A3SS	4933	adipose_subcutaneous	4	Adipose (sc)	GREB1L	ENSG00000141449.14	chr18	1.2793173	2.115005	7.123138	3.052112e-12	8.545914e-09	16.692429
4	RHCG-1776	A3SS	1776	adipose_subcutaneous	4	Adipose (sc)	RHCG	ENSG00000140519.14	chr15	-0.6930009	1.636472	-3.922124	9.797866e-05	3.919146e-02	1.142232
5	XIST-2253	A3SS	2253	adipose_visceral_omentum	12	Adipose (v)	XIST	ENSG00000229807.11	chrX	-4.4403352	3.113532	-33.950800	2.654474e-123	2.209585e-119	241.826117
6	XIST-2252	A3SS	2252	adipose_visceral_omentum	12	Adipose (v)	XIST	ENSG00000229807.11	chrX	-2.4506832	3.650617	-18.890779	2.817671e-58	1.172715e-54	114.731682
</pre>
There are 2887 significant events in the file.

In [ ]:
glimpse(gene_as)
gene_as$Tissue <- factor(gene_as$Tissue)
length(levels(gene_as$Tissue))
table(is.na(gene_as$Display))
table(gene_as$Display)
colnames(gene_as)
head(gene_as)
tissue_reduction$display_name <- factor(tissue_reduction$display_name)

In [50]:
x_as_events <- gene_as[gene_as$chr=="chrX",]
message("There were ",nrow(gene_as)," total significant alternative splicing events (gene_as)")
message("There were ",nrow(x_as_events)," total significant alternative splicing events on the X chromosome (gene_as)")
message("i.e., ", (100*nrow(x_as_events)/nrow(gene_as)), "% of all significant AS events were on the X chromosome")

numberOfUniqueTissues <- length(summary(as.factor(gene_as$Display),maxsum=500))
numberOfASEmechanisms <- length(summary(as.factor(gene_as$ASE),maxsum=500))

message("gene_as now has ",numberOfUniqueTissues, " tissues and ", numberOfASEmechanisms, " ASE categories")
message("ASE:")
summary(as.factor(gene_as$ASE),maxsum=500)

There were 6370 total significant alternative splicing events (gene_as)

There were 783 total significant alternative splicing events on the X chromosome (gene_as)

i.e., 12.2919937205651% of all significant AS events were on the X chromosome

gene_as now has 39 tissues and 5 ASE categories

ASE:



A3SS A5SS  MXE   RI   SE 
 693  421  199  418 4639

### 3.2 gene_dge.tsv

This file contains (description)
Here is a typical line
<pre>
Tissue  ENSG_ver        ENSG_no_ver     GeneSymbol      counts  Display logFC   AveExpr t       PValue  AdjPVal B
1       adipose_subcutaneous    ENSG00000176728.7       ENSG00000176728 TTTY14  765     Adipose (sc)    -7.98216577151896     -0.928812923511535       -139.823010017733       0       0       1107.42326360464
2       adipose_subcutaneous    ENSG00000231535.5       ENSG00000231535 LINC00278       765     Adipose (sc)    -6.09542040758638      -2.77656379347601       -126.913818678612       0       0       1050.36559888639
3       adipose_subcutaneous    ENSG00000129824.15      ENSG00000129824 RPS4Y1  765     Adipose (sc)    -9.6641901864726      4.63528767282141 -125.827094717734       0       0       1041.87660796556
4       adipose_subcutaneous    ENSG00000067646.11      ENSG00000067646 ZFY     765     Adipose (sc)    -9.50458982938477     0.672755457406984        -125.037143030325       0       0       1033.61131617113
5       adipose_subcutaneous    ENSG00000229807.10      ENSG00000229807 XIST    765     Adipose (sc)    9.89280986473167      1.23756039627052 121.69689757218 0       0       1030.17757492281
6       adipose_subcutaneous    ENSG00000229236.1       ENSG00000229236 TTTY10  765     Adipose (sc)    -6.20901295440725     -2.74524363170072        -122.540297482165       0       0       1029.41424065532
7       adipose_subcutaneous    ENSG00000233864.7       ENSG00000233864 TTTY15  765     Adipose (sc)    -8.19361688496523     -0.741097276206495       -122.47199454746        0       0       1027.58175958647
8       adipose_subcutaneous    ENSG00000260197.1       ENSG00000260197 AC010889.1      765     Adipose (sc)    -8.52835806068555      -0.686009457030557      -119.790486729538       0       0       1015.85291786821
9       adipose_subcutaneous    ENSG00000183878.15      ENSG00000183878 UTY     765     Adipose (sc)    -9.52139275438866     1.60375445153084 -110.599936868261       0       0       953.069478992754
</pre>
There are 7417 significant events in the file.

In [49]:
x_dge_events <- gene_dge[gene_dge$chr=="chrX",]
message("There were ",nrow(gene_dge)," total significant differential gene expression events (gene_dge)")
message("There were ",nrow(x_dge_events)," total significant differential gene expression events on the X chromosome (gene_as)")
message("i.e., ", (100*nrow(x_dge_events)/nrow(gene_dge)), "% of all significant DGE events were on the X chromosome")

numberOfUniqueTissues <- length(summary(as.factor(gene_dge$Display),maxsum=500))

message("gene_dge now has ",numberOfUniqueTissues, " tissues")

There were 12633 total significant differential gene expression events (gene_dge)

There were 774 total significant differential gene expression events on the X chromosome (gene_as)

i.e., 6.12681073379245% of all significant DGE events were on the X chromosome

gene_dge now has 39 tissues



### 3.3 Count events by chromosome

Count the number of significant alternative splicing events per chromosome and save to the file **Total_AS_by_chr.tsv**.

### 3.3.1 by alternative splicing events

In [51]:
total_as_by_chr <- gene_as          %>% 
                   group_by(chr)    %>% 
                   count(chr)       %>% 
                   arrange(desc(n)) %>% 
                   as.data.frame()
total_as_by_chr$chr <- factor(total_as_by_chr$chr, levels = total_as_by_chr$chr)
length(total_as_by_chr$chr)
total_as_by_chr
glimpse(total_as_by_chr)
write.table(total_as_by_chr, file= "../data/Total_AS_by_chr.tsv", sep="\t", quote = FALSE, row.names=F)

[1] 23

chr,n
<fct>,<int>
chrX,783
chr1,636
chr19,442
chr11,374
chr2,374
chr3,368
chr17,363
chr12,334
chr16,313


Observations: 23
Variables: 2
$ chr <fct> chrX, chr1, chr19, chr11, chr2, chr3, chr17, chr12, chr16, chr4, …
$ n   <int> 783, 636, 442, 374, 374, 368, 363, 334, 313, 292, 248, 243, 225, …


### 3.3.2 by gene expression

In [52]:
total_dge_by_chr <- gene_dge          %>% 
                   group_by(chr)    %>% 
                   count(chr)       %>% 
                   arrange(desc(n)) %>% 
                   as.data.frame()
total_dge_by_chr$chr <- factor(total_dge_by_chr$chr, levels = total_dge_by_chr$chr)
length(total_dge_by_chr$chr)
total_dge_by_chr
glimpse(total_dge_by_chr)
write.table(total_dge_by_chr, file= "../data/Total_DGE_by_chr.tsv", sep="\t", quote = FALSE, row.names=F)

[1] 26

chr,n
<fct>,<int>
chrY,1693
chr1,945
chr2,897
chrX,774
chr14,554
chr12,547
chr6,511
chr11,501
chr3,486


Observations: 26
Variables: 2
$ chr <fct> chrY, chr1, chr2, chrX, chr14, chr12, chr6, chr11, chr3, chr19, c…
$ n   <int> 1693, 945, 897, 774, 554, 547, 511, 501, 486, 478, 464, 464, 443,…


### 3.4 Count events by genes 

### 3.4.1 by alternative splicing

In [91]:
total_as_by_geneSymbol <- gene_as %>% 
                          group_by(GeneSymbol) %>% 
                          count(GeneSymbol)    %>% 
                          arrange(desc(n))     %>% 
                          as.data.frame()
total_as_by_geneSymbol$GeneSymbol <- factor(total_as_by_geneSymbol$GeneSymbol, 
                                            levels = total_as_by_geneSymbol$GeneSymbol)
length(total_as_by_geneSymbol$GeneSymbol)
head(total_as_by_geneSymbol,10)
write.table(total_as_by_geneSymbol, file = "../data/Total_AS_by_geneSymbol.tsv", sep = "\t", quote=FALSE, row.names = F)

[1] 2887

,GeneSymbol,n
,<fct>,<int>
1,XIST,342
2,DDX3X,97
3,KDM5C,47
4,ZFX,47
5,KDM6A,27
6,UCA1,25
7,MYB,23
8,NLRP2,22
9,NDRG4,20


### 3.5 Count most frequent splicing by tissue

### 3.5.1 by alternative splicing

In [87]:
total_as_by_tissue <- gene_as %>% 
                      group_by(Display) %>% 
                      count(Display)    %>% 
                      arrange(desc(n))  %>% 
                      as.data.frame()
total_as_by_tissue$Display <- factor(total_as_by_tissue$Display, 
                                     levels = total_as_by_tissue$Display)
head(total_as_by_tissue,10)
length(total_as_by_tissue$Display)
write.table(total_as_by_tissue, file = "../data/Total_AS_by_tissue.tsv", sep = "\t", row.names = F)

,Display,n
,<fct>,<int>
1,Breast,4361
2,Nucleus accumbens,335
3,Esophagus (mu),261
4,Aorta,139
5,Thyroid,104
6,Fibroblasts,97
7,Skeletal muscle,80
8,Adipose (sc),78
9,Pituitary,65


[1] 39

### 3.5.2 by gene expression

In [90]:
#glimpse(gene_dge)
gene_dge$GeneSymbol <- factor(gene_dge$GeneSymbol)
total_dge_by_tissue <- gene_dge %>% 
                          select(c(GeneSymbol, Display, logFC)) %>%
                          group_by(Display) %>%
                          arrange(desc(logFC)) %>%
                          tally() %>%
                          arrange(desc(n)) %>%
                          as.data.frame()
head(total_dge_by_tissue,10)
length(total_dge_by_tissue$Display)
write.table(total_dge_by_tissue, file = "../data/Total_DGE_by_tissue.tsv", sep = "\t", quote=FALSE, row.names = F)

,Display,n
,<fct>,<int>
1,Breast,5433
2,Adipose (sc),765
3,Pituitary,682
4,Thyroid,486
5,Skin (not exposed),435
6,Skin (exposed),331
7,Adipose (v),325
8,Skeletal muscle,296
9,Left ventricle,274


[1] 39

###  3.6 Significant Count by splicing type 
We define **significant** to be FC > 1.5 and pVal < 0.05

Our starting values were the significant events, all meeting the criteria FC > 1.5 and pVal < 0.05


In [ ]:
total_as_by_splicingtype <- gene_as %>% 
                            group_by(ASE)    %>% 
                            count(ASE)       %>% 
                            arrange(desc(n)) %>%
                            as.data.frame()
total_as_by_splicingtype$ASE <- factor(total_as_by_splicingtype$ASE, levels = total_as_by_splicingtype$ASE)
total_as_by_splicingtype
write.table(total_as_by_splicingtype, file= "../data/Total_AS_by_splicingtype.tsv")

###  3.7 Significant Count by splicing type (significant == FC > 1.5 and pVal < 0.05)

In [ ]:
A3SS_keep <- as.character(gene_as$ASE) %in% "A3SS"
table(A3SS_keep)
A3SS.gene_as <- data.frame(gene_as[A3SS_keep == TRUE,])

A5SS_keep <- as.character(gene_as$ASE) %in% "A5SS"
table(A5SS_keep)
A5SS.gene_as <- data.frame(gene_as[A5SS_keep == TRUE,])

MXE_keep  <- as.character(gene_as$ASE) %in% "MXE"
table(MXE_keep)
MXE.gene_as <- data.frame(gene_as[MXE_keep == TRUE,])

SE_keep   <- as.character(gene_as$ASE) %in% "SE"
table(SE_keep)
SE.gene_as <- data.frame(gene_as[SE_keep == TRUE,])

RI_keep   <- as.character(gene_as$ASE) %in% "RI"
table(RI_keep)
RI.gene_as <- data.frame(gene_as[RI_keep == TRUE,])

dim(A3SS.gene_as)
dim(A5SS.gene_as)
dim(MXE.gene_as)
dim(SE.gene_as)
dim(RI.gene_as)


### 3.8 Siginficant spliced by Gene for each splicing factor

In [ ]:
A3SS.res <- A3SS.gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
A3SS.res$GeneSymbol <- factor(A3SS.res$GeneSymbol, levels = A3SS.res$GeneSymbol)
message("Significant spliced genes for A3SS\n",
        paste(length(A3SS.res$GeneSymbol)), collapse=" ")
head(A3SS.res)

A5SS.res <- A5SS.gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
A5SS.res$GeneSymbol <- factor(A5SS.res$GeneSymbol, levels = A5SS.res$GeneSymbol)
message("Significant spliced genes for A5SS\n",
        paste(length(A5SS.res$GeneSymbol)), collapse=" ")
head(A5SS.res)

MXE.res <- MXE.gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
MXE.res$GeneSymbol <- factor(MXE.res$GeneSymbol, levels = MXE.res$GeneSymbol)
message("Significant spliced genes for MXE\n",
        paste(length(MXE.res$GeneSymbol)), collapse=" ")
head(MXE.res)

RI.res <- RI.gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
RI.res$GeneSymbol <- factor(RI.res$GeneSymbol, levels = RI.res$GeneSymbol)
message("Significant spliced genes for RI\n",
        paste(length(RI.res$GeneSymbol)), collapse=" ")
head(RI.res)

SE.res <- SE.gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
SE.res$GeneSymbol <- factor(SE.res$GeneSymbol, levels = SE.res$GeneSymbol)
message("Significant spliced genes for SE\n",
        paste(length(SE.res$GeneSymbol)), collapse=" ")
head(SE.res)

### 3.9 Count most frequent spliced genes

In [ ]:
genesMostFrequentlySpliced <- gene_as %>% 
                              group_by(GeneSymbol) %>% 
                              count(GeneSymbol)    %>% 
                              arrange(desc(n))     %>% 
                              as.data.frame()
genesMostFrequentlySpliced$GeneSymbol <- factor(genesMostFrequentlySpliced$GeneSymbol, 
                                                levels = genesMostFrequentlySpliced$GeneSymbol)
length(genesMostFrequentlySpliced$GeneSymbol)

#Add number of tissues
nTissues <- rep(NA, length(genesMostFrequentlySpliced))
for (i in 1:nrow(genesMostFrequentlySpliced)) {
    df_gene <- gene_as %>% 
               filter(GeneSymbol == genesMostFrequentlySpliced$GeneSymbol[i])
    nTissues[i] <- length(unique(df_gene$Tissue))
}
genesMostFrequentlySpliced$Tissues <- nTissues
head(genesMostFrequentlySpliced)
write.table(genesMostFrequentlySpliced, file = "../data/genesWithCommonAS.tsv", sep = "\t", quote = F, row.names = F)

### 3.10 Count most frequent spliced chromosomes
To get an indication of which chromosome has the most frequent slicing event (regardless of type)
We create an index based upon the number of exons per chromosome.

get the annotation file, at this writing, gencode.v30.annotation.gtf
The information as to the number of exons within the chromosome may be found there

In [ ]:
exons <- gencode[ gencode$type == "exon", ]
exons <- as.data.frame(exons)

#Obtain chromosomes we have splicing information for (recall we did not use chr Y in our analysis)
all_chr <- as.character(unique(gene_as$chr))
chr_counts <- rep(0, length(all_chr))


for (i in 1:length(all_chr)) {
  chr_counts[i] <- nrow(exons[exons$seqnames == all_chr[i], ])
}

exon_counts <- data.frame(chr = all_chr, counts = chr_counts)

# Count most frequent spliced chromosomes
res <- gene_as %>% group_by(chr) %>% count(chr) %>% arrange(desc(n)) %>% as.data.frame()
res$chr <- factor(res$chr, levels = res$chr)

idx <- match(res$chr, exon_counts$chr)

res$ExonCounts <- exon_counts$counts[idx]

res$Index <- (res$n / res$ExonCounts) * 1000

res_sorted <- res %>% arrange(desc(Index))
res_sorted$chr <- factor(res_sorted$chr, levels = res_sorted$chr)
glimpse(res_sorted)
write.table(res_sorted, file = "../data/SplicingIndex_chr.tsv", sep = "\t", quote = F, row.names = F)

### 3.11 Overlap between Differential Gene Expression and Differential Alternative Splicing

First gather the data

In [ ]:
total_AS_Genes <- read.table(file="../data/Total_AS_by_geneSymbol.tsv", header=TRUE, sep="\t",
                               skipNul=FALSE, stringsAsFactors = FALSE)
sigAsGenes <- sort(total_AS_Genes$GeneSymbol)
dge <- read.table("../data/gene_dge.tsv", sep = "\t", header = FALSE, row.names=1, skip = 1)
dge_genes <- sort(dge$V5)
head(dge_genes)
all_genes_data <- read.table("../assets/all_gene_as_gene_names.csv")
names(all_genes_data) <- c("GeneSymbol", "ensg")
all_genes <- sort(all_genes_data$GeneSymbol)

In [ ]:
head(all_genes)

### 3.12 We then do a hypergeometric/Fisher test to look for overrepresentation
The universe consists of all genes with at least one read (all_genes_data).
So we have

|  	|  DGE+| DGE-|
|-	|-	|-	|
| DAS+|  a|  b|
| DAS-|  c| d|

In [ ]:
message("Number of sigAsGenes ", length(sigAsGenes))
notSigAs <- setdiff(all_genes,sigAsGenes)
message("Number of genes that are NOT sigAs ", length(notSigAs))
message("Number of DGE genes", length(dge_genes))
notDGE <- setdiff(all_genes,dge_genes)
message("Number of genes that are NOT DGE ", length(notDGE))
a <- intersect(sigAsGenes, dge_genes)
b <- intersect(sigAsGenes, notDGE)
c <- intersect(notSigAs, dge_genes)
d <- intersect(notSigAs, notDGE)
message("a: ", length(a), "; b: ",  length(b), "; c: ",  length(c), "; d: ",  length(d))

In [ ]:
m <- matrix(c(length(a),length(b),length(c),length(d)), nrow=2,byrow = TRUE)
fisher.test(m)

### Appendix - Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **'artefacts'**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### Appendix 1. Checksums with the sha256 algorithm

In [ ]:
rm (notebookid)
notebookid   = "countGenesAndEvents"
notebookid

message("Generating sha256 checksums of the file `../data/gene_as.tsv` directory .. ")
system(paste0("cd ../data && find . -name gene_as.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

message("Generating sha256 checksums of the file `../data/all_gene_as.tsv` directory .. ")
system(paste0("cd ../data && find . -name gene_dge.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

message("Generating sha256 checksums of the file `../data/gene_dge.tsv` directory .. ")
system(paste0("cd ../data && find . -name gene_as.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

message("Generating sha256 checksums of the file `../data/Total_AS_by_chr.tsv` directory .. ")
system(paste0("cd ../data && find . -name Total_AS_by_chr.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

message("Generating sha256 checksums of the file `../data/Total_AS_by_geneSymbol.tsv` directory .. ")
system(paste0("cd ../data && find . -name Total_AS_by_geneSymbol.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

message("Generating sha256 checksums of the file `../data/Total_AS_by_tissue.tsv` directory .. ")
system(paste0("cd ../data && find . -name Total_AS_by_tissue.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

message("Generating sha256 checksums of the file `../data/Total_AS_by_splicingtype.tsv` directory .. ")
system(paste0("cd ../data && find . -name Total_AS_by_splicingtype.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

message("Generating sha256 checksums of the file `../data/genesWithCommonAS.tsv` directory .. ")
system(paste0("cd ../data && find . -name genesWithCommonAS.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

message("Generating sha256 checksums of the file `../data/SplicingIndex_chr.tsv` directory .. ")
system(paste0("cd ../data && find . -name SplicingIndex_chr.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")


### Appendix 2. Libraries metadata

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", notebookid, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", notebookid ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]